In [188]:
# library imports
import librosa
import torch as tc
import numpy as np
import pandas as pd
from torch import nn
import seaborn as sns
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn import preprocessing

**Superficial data analysis**

In [189]:
# data upload
dataPath = '../input/random-testing-data1/features_30_sec_randEdit.csv'
audioData = pd.read_csv(dataPath)
audioData[:10]

In [190]:
audioData.dtypes

**Correlation testing**

In [191]:
# general correlogram 
corrDataFrame = audioData.corr(method ='pearson')
plt.figure(figsize = (20,20))
sns.heatmap(corrDataFrame, cmap="Blues")

In [192]:
# correlogram filtering out only pairs with a correlation coefficient of greater than 0.8 or lesser than -0.8
corrDataFrame = audioData.corr(method ='pearson')
corrDataFrame[np.abs(corrDataFrame) < 0.8] = 0

mask = np.zeros_like(corrDataFrame, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize = (20,20))
sns.heatmap(corrDataFrame, cmap="coolwarm", mask=mask)

In [193]:
# filtering only the pairs which have high correlation

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

topCorr = get_top_abs_correlations(corrDataFrame, 221) # 221 as the 221st entry was the last one greater than 0.8
topCorr = pd.DataFrame(topCorr)
pd.set_option('display.max_rows', None)

# print("Top Absolute Correlations")
# topCorr

In [194]:
'''

Notes :

- Use Ctrl + F to find the rows with the desired dependent variable.
- Consider only the elements paired with them as independent variables for the model.

'''

**Model training**

In [195]:
# train/test data loading
df = pd.read_csv(dataPath)
train = df.sample(frac=0.5,random_state=100) # random state is a seed value
test = df.drop(train.index)

# dependent variable, categorical (0-100) (one hot encoded)
Ytest = test['volume']
Ytest = pd.get_dummies(Ytest)
Ytest = Ytest.to_numpy()

Ytrain = train['volume']
Ytrain = pd.get_dummies(Ytrain)
Ytrain = Ytrain.to_numpy()

# independent variable 1, categorical (label encoded)
# note : [blues:0, classical:1, country:2, disco:3, hiphop:4, jazz:5, metal:6, pop:7, reggae:8, rock:9]
le = preprocessing.LabelEncoder()
X1test = le.fit_transform(test['label'])
X1train = le.fit_transform(train['label'])

# independent variable 2, numerical
X2test = test['chroma_stft_mean'].to_numpy()
X2train = train['chroma_stft_mean'].to_numpy()

# add on the remaining relevant variables here

In [196]:
# declaring constants
numInputs = 2 # number of input variables based on correlation test
Ntest = 500 # number of testing entries
Ntrain = 500 # number of training entries

In [197]:
# data reshaping
Xtest = np.dstack([X1test,X2test]) # add on all the independent testing variables in this list
Xtest = Xtest.reshape(Ntest,numInputs) 

Xtrain = np.dstack([X1train,X2train]) # add on all the independent training variables in this list
Xtrain = Xtrain.reshape(Ntrain,numInputs) 

if (Ytest.shape[1] == Ytrain.shape[1]) :
    
    print(f'Number of classes : {Ytest.shape[1]}\n')
    # declaring number of outputs based on the results of one hot encoding
    numOutputs = 99
    
else :
    
    print('Re-execute test/train split\n')
    
Ytest = Ytest.reshape(Ntest, 99) # one dependent test variable
Ytrain = Ytrain.reshape(Ntrain, 99) # one dependent train variable

# data type conversion
xtest = tc.from_numpy(Xtest.astype(np.float32))
xtrain = tc.from_numpy(Xtrain.astype(np.float32))

ytest = tc.from_numpy(Ytest.astype(np.float32))
ytrain = tc.from_numpy(Ytrain.astype(np.float32))


In [198]:
# model definition
class NeuralNetwork(nn.Module):
    def __init__(self, numInputs, numOutputs):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(numInputs, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, numOutputs),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [199]:
# error function & optimizer
model = NeuralNetwork(numInputs, numOutputs)
e_func = tc.nn.MSELoss()
optim = tc.optim.SGD(model.parameters(), lr = 0.001)

In [200]:
# move model to GPU
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
model.to(device)

In [205]:
# model training/testing loop
ep = 1000
train_losses, test_losses, accuracies = [], [], []

for e in range(ep+1) :
    
    # training model
    running_loss = 0
    xtrain, ytrain = xtrain.to(device), ytrain.to(device)
    
    optim.zero_grad() 
    
    output = model(xtrain) 
    loss = e_func(output, ytrain)
    
    loss.backward() 
    optim.step() 
    
    running_loss += loss.item()
    train_losses.append(running_loss/len(ytrain))
    
    # testing model
    test_loss, accuracy = 0, 0
    
    with tc.no_grad():
            
        model.eval()
        xtest, ytest = xtest.to(device), ytest.to(device)
        output = model(xtest) 
        test_loss += e_func(output, ytest)
        
        _, predictions = tc.max(output, 1)
        _, targets = tc.max(ytest, 1)
        accuracy += (predictions == targets).sum().item()
        
        test_losses.append(test_loss/len(ytest))
        accuracies.append(accuracy/len(ytest))
        model.train()

    print(f'Epoch: {e}/{ep}\n',
          f'Training loss: {running_loss/len(ytrain)}\n',
          f'Test loss: {test_loss/len(ytest)}\n',
          f'Accuracy: {accuracy/len(ytest)}\n')

In [202]:
# plot train & test loss per iteration
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

In [203]:
# plot testing accuracy
plt.plot(accuracies, label='Model accuracy')
plt.legend(frameon=False)

In [204]:
# save model
PATH = '../output/model.pth' # input directory in which the model is to be saved
tc.save(model, PATH)

**Testing model deployment**

In [ ]:
# model class
class NeuralNetwork(nn.Module):
    def __init__(self, numInputs, numOutputs):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(numInputs, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, numOutputs),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# load model
PATH = '../output/model.pth' # directory in which the model is saved
model = tc.load(PATH)
model.eval()

In [ ]:
# loading sample file for testing
fname = '../input/test-audio/Kalimba.mp3'
genre = '' # specify genre
SR = 22050
data, _ = librosa.load(fname, sr=SR, mono=True)

# visualizing sample mp3
plt.figure(figsize = (16, 6))
librosa.display.waveplot(y = data, sr = 22050, color = "#A300F9")
plt.title("Sound Waves in sample", fontsize = 10)

In [ ]:
# function to take an mp3 file as a parameter and return predicted volume by calling saved model
def predictVolume(data) :
    
    # getting all necessary model input values in the correct data format
    
    return prediction